# Test : Matrices vs Cache

Pour tester le pouvoir des caches, prenons de grosses matrices et faisons des calculs. Des matrices assez grosses pour ne pas pouvoir être dans le cache.

Et juste, observons comment l'ordre des calculs entraine des différences de temps de calcul selon que l'ordre des calculs est plus ou moins compatible avec le cache. 

Pour faire les calculs, juste multiplions les 2 matrices entre elles. C'est un programme avec une triple boucle imbriquée :
* une boucle pour les lignes
* une boucle pour les colonnes
* une boucle intermédiaire (K)

On peut aranger les 3 boucles dans l'ordre que l'on veut, il y aura toujours le même nombre de calculs.

Version Lignes-K-Colonnes :

In [1]:
%%writefile multiplicationMatriceLiKCol.c
#include <stdio.h>
#include <stdlib.h>

int a[1000][1000], b[1000][1000], 
  c[1000][1000], i, li, co, k, s ;
int main (int argc, char * argv[]) {
  if (argc==1)    {i=1;}
  else {i=atoi(argv[1]);}
  for(;i;i--) {
    for(co=0;co<1000;co++) {
      for(li=0;li<1000;li++) {
        a[li][co]=1;b[li][co]=1;c[li][co]=0;}}
    for (li=0;li<1000;li++) {
      for (k=0;k<1000;k++) {
        s = a[li][k];
        for (co=0;co<1000;co++) {
          c[li][co] += s*b[k][co];}}}}
  printf("fin\n");
  return 0;}

Overwriting multiplicationMatriceLiKCol.c


In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static multiplicationMatriceLiKCol.c
cat multiplicationMatriceLiKCol.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"multiplicationMatriceLiKCol.c"
	.comm	a,4000000,4
	.comm	b,4000000,4
	.comm	c,4000000,4
	.comm	i,4,4
	.comm	li,4,4
	.comm	co,4,4
	.comm	k,4,4
	.comm	s,4,4
	.section	.rodata
	.align	2
.LC0:
	.ascii	"fin\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {r4, fp, lr}
	add	fp, sp, #8
	sub	sp, sp, #12
	str	r0, [fp, #-16]
	str	r1, [fp, #-20]
	ldr	r3, [fp, #-16]
	cmp	r3, #1
	bne	.L2
	ldr	r3, .L17
	mov	r2, #1
	str	r2, [r3, #0]
	b	.L4
.L2:
	ldr	r3, [fp, #-20]
	add	r3, r3, #4
	ldr	r3, [r3, #0]
	mov	r0, r3
	bl	atoi
	mov	r2, r0
	ldr	r3, .L17
	str	r2, [r3, #0]
	b	.L4
.L15:
	ldr	r3, .L17+4
	mov	r2, #0
	str	r2, [r3, #0]
	b	.L5
.L8:
	ldr	r3, .L17+8
	mov	r2,

In [ ]:
%%sh
arm-linux-gnueabi-gcc -static multiplicationMatriceLiKCol.c
time qemu-arm a.out 1

Version Colonne-K-Ligne :

In [1]:
%%writefile multiplicationMatriceColKLi.c
#include <stdio.h>
#include <stdlib.h>

int a[1000][1000], b[1000][1000], 
  c[1000][1000], i, li, co, k, s ;
int main (int argc, char * argv[]) {
  if (argc==1)    {i=1;}
  else {i=atoi(argv[1]);}
  for(;i;i--) {
    for(co=0;co<1000;co++) {
      for(li=0;li<1000;li++) {
        a[li][co]=1; b[li][co]=1;c[li][co]=0;}}
    for (co=0;co<1000;co++) {
      for (k=0;k<1000;k++) {
        s = b[k][co];
        for (li=0;li<1000;li++) {
          c[li][co] += a[li][k]*s;}}}}
  printf("fin\n");
  return 0;}

Overwriting multiplicationMatriceColKLi.c


In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static multiplicationMatriceColKLi.c
cat multiplicationMatriceColKLi.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"multiplicationMatriceColKLi.c"
	.comm	a,4000000,4
	.comm	b,4000000,4
	.comm	c,4000000,4
	.comm	i,4,4
	.comm	li,4,4
	.comm	co,4,4
	.comm	k,4,4
	.comm	s,4,4
	.section	.rodata
	.align	2
.LC0:
	.ascii	"fin\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {r4, fp, lr}
	add	fp, sp, #8
	sub	sp, sp, #12
	str	r0, [fp, #-16]
	str	r1, [fp, #-20]
	ldr	r3, [fp, #-16]
	cmp	r3, #1
	bne	.L2
	ldr	r3, .L17
	mov	r2, #1
	str	r2, [r3, #0]
	b	.L4
.L2:
	ldr	r3, [fp, #-20]
	add	r3, r3, #4
	ldr	r3, [r3, #0]
	mov	r0, r3
	bl	atoi
	mov	r2, r0
	ldr	r3, .L17
	str	r2, [r3, #0]
	b	.L4
.L15:
	ldr	r3, .L17+4
	mov	r2, #0
	str	r2, [r3, #0]
	b	.L5
.L8:
	ldr	r3, .L17+8
	mov	r2,

In [3]:
%%sh
arm-linux-gnueabi-gcc -static multiplicationMatriceColKLi.c
time qemu-arm a.out 1

fin


42.81user 0.04system 0:44.40elapsed 96%CPU (0avgtext+0avgdata 15872maxresident)k
0inputs+0outputs (0major+4059minor)pagefaults 0swaps


## Arm vs X86 ?

Mais est-ce que qemu simule le cache (ou seulement bénéficie d'un cache de la machine ? ou produit un code qui peut en bénéficiér ?) ? Bref, avec qemu, ce n'est pas tout à fait sûr de savoir ce qui se passe. Pour avoir confirmation de ce que l'on a l'impression d'observer regardons en X86 (cette machine est en X86, pas en ARM) pour un langage compilé (il ne faudrait peut-être pas prendre Python ou java pour voir cela)

In [4]:
%%sh
gcc multiplicationMatriceLiKCol.c
time ./a.out 1

fin


4.12user 0.00system 0:04.28elapsed 96%CPU (0avgtext+0avgdata 12208maxresident)k
0inputs+0outputs (0major+3088minor)pagefaults 0swaps


In [5]:
%%sh
gcc multiplicationMatriceColKLi.c
time ./a.out 1

fin


16.40user 0.00system 0:17.20elapsed 95%CPU (0avgtext+0avgdata 12204maxresident)k
0inputs+0outputs (0major+3088minor)pagefaults 0swaps
